In [2]:
import json
from datasets import load_dataset, Dataset
# Specify the path to the JSON file
file_path = 'squad-sr-lat.json'

# Open the JSON file and load its contents
with open(file_path, 'r', encoding='utf8') as file:
    data = json.load(file)

# Now, 'data' contains the parsed JSON data as a Python dictionary
print(data.keys())


dict_keys(['version', 'data'])


In [3]:
sentence1 = []
sentence2 = []
data_for_eval = {
    'id': [],
    'title': [],
    'context': [],
    'question': [],
    'answers': []
}
i = 1
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            data_for_answer = {
                "text": [],
                'answer_start': []
            }
            for answer in qa_pair['answers']:
                short_answer = answer['text']
                answer_start = answer['answer_start']
                sentence1.append(question)
                sentence2.append(short_answer)
                data_for_answer['text'].append(short_answer)
                data_for_answer['answer_start'].append(answer_start)
            data_for_eval['id'].append(str(i))
            data_for_eval['title'].append(title)
            data_for_eval['context'].append(context)
            data_for_eval['question'].append(question)
            data_for_eval['answers'].append(data_for_answer)
            i += 1
                # print(answer)
dataset_for_eval = Dataset.from_dict(data_for_eval)

In [8]:
# from evaluate import evaluator
# task_evaluator = evaluator("question-answering")
# # data = load_dataset("squad_v2", split="validation[:2]")
# results = task_evaluator.compute(
#     model_or_pipeline="classla/bcms-bertic",
#     data=dataset_for_eval,
#     metric="squad_v2",
#     squad_v2_format=True,
# )

In [ ]:
# results

{'exact': 0.0,
 'f1': 4.2324445484296795,
 'total': 269,
 'HasAns_exact': 0.0,
 'HasAns_f1': 4.2324445484296795,
 'HasAns_total': 269,
 'best_exact': 0.0,
 'best_exact_thresh': 0.0,
 'best_f1': 4.232444548429679,
 'best_f1_thresh': 0.00017354990995954722,
 'total_time_in_seconds': 56.48018586899707,
 'samples_per_second': 4.762732201765976,
 'latency_in_seconds': 0.20996351624162482}

loading the data to wanted format

In [9]:
i = 1
titles = []
contexts = []
queryes = []
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        currect_queryes = []
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            
            currect_queryes.append(question)
            i += 1
        titles.append(title)
        contexts.append(context)
        queryes.append(currect_queryes)
                # print(answer)
dataset_for_eval = Dataset.from_dict(data_for_eval)

In [11]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queryes})

dataset_for_eval_df.head()
dataset_for_eval_df['queries'][0]

['Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 'Šta je ispred Glavne zgrade Notr Dame?',
 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 'Šta je Grota u Notr Dami?',
 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?']

In [14]:
dataset_for_eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18896 entries, 0 to 18895
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    18896 non-null  object
 1   context  18896 non-null  object
 2   queries  18896 non-null  object
dtypes: object(3)
memory usage: 443.0+ KB


In [15]:
data_mini = dataset_for_eval_df[:10]
data_mini.head()

,title,context,queries
0,University_of_Notre_Dame,"Arhitektonski, škola ima katolički karakter. N...","[Kome se u 1858. godini u Lurdu, Francuska, na..."
1,University_of_Notre_Dame,"Kao i na većini drugih univerziteta, studenti ...",[Kada je počelo izdavanje Scholastičkog časopi...
2,University_of_Notre_Dame,Univerzitet je glavna sedište Kongregacije Sve...,"[Gde je sedište Kongregacije Svetog Krsta?, Št..."
3,University_of_Notre_Dame,Koledž za inženjerstvo osnovan je 1920. godine...,[Koliko diploma na nivo BS se nudi u Koledžu z...
4,University_of_Notre_Dame,Svi studenti Notr Dame-a su deo jednog od pet ...,[Koje su subjekti koje pružaju pomoć u upravlj...


In [60]:
type(data_mini['queries'][0])

list

In [16]:
data_mini.drop(columns=['title'], inplace=True)
data_mini.head()

/tmp/ipykernel_11934/1436900563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_mini.drop(columns=['title'], inplace=True)


,context,queries
0,"Arhitektonski, škola ima katolički karakter. N...","[Kome se u 1858. godini u Lurdu, Francuska, na..."
1,"Kao i na većini drugih univerziteta, studenti ...",[Kada je počelo izdavanje Scholastičkog časopi...
2,Univerzitet je glavna sedište Kongregacije Sve...,"[Gde je sedište Kongregacije Svetog Krsta?, Št..."
3,Koledž za inženjerstvo osnovan je 1920. godine...,[Koliko diploma na nivo BS se nudi u Koledžu z...
4,Svi studenti Notr Dame-a su deo jednog od pet ...,[Koje su subjekti koje pružaju pomoć u upravlj...


In [22]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, pipeline
import pandas as pd
import numpy as np

# Load model and tokenizer
model_name = "classla/bcms-bertic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/home/teodora/Development/Embedding-Tutorial/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [1]:
import sentence_transformers



/home/teodora/Development/Embedding-Tutorial/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [23]:
# Generate embeddings
import torch

def get_sentence_embedding(sentence):
    encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
    attention_mask = encoded_input['attention_mask']
    # Get the model output (without the specific classification head)
    with torch.no_grad():
        output = model(**encoded_input)

    token_embeddings = output.last_hidden_state # poslednji nivo
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

    # mean pooling operation, considering the BERT input_mask and padding
    sentence_embedding = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    return sentence_embedding.flatten().tolist()

making the pipieline for all models

In [26]:
context_embeddings = []

for text in data_mini['context']:
    text_embedding = get_sentence_embedding(text)
    context_embeddings.append(text_embedding)

print(context_embeddings)
print(len(context_embeddings))
print(len(text_embedding))

[[0.18937091529369354, 0.044879719614982605, 0.0154149504378438, 0.04394680634140968, 0.01704881340265274, -0.06912563741207123, 0.09799977391958237, 0.2453484982252121, -0.045774590224027634, -0.0057826596312224865, 0.05354193225502968, -0.055794745683670044, 0.05333784967660904, 0.07620985805988312, 0.13949593901634216, 0.20684367418289185, -0.034277237951755524, 0.11561574041843414, 0.03756776824593544, -0.025886530056595802, -0.06385035067796707, 0.01110877189785242, 0.14294104278087616, 0.08407535403966904, -0.03739964962005615, -0.0526822991669178, 0.026381846517324448, 0.0998023971915245, 0.005950169637799263, 0.109789177775383, 0.10341539233922958, 0.11645758897066116, -0.003400396788492799, 0.04540389031171799, 0.1476154923439026, 0.06176649406552315, 0.27203622460365295, -0.4186108112335205, -0.021751701831817627, 0.07747364044189453, 0.08159049600362778, 0.05826033651828766, 0.02513401210308075, 0.025953469797968864, 0.13207001984119415, 0.16606047749519348, -0.0033698482438

In [36]:
data_mini['queries'][0]

['Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 'Šta je ispred Glavne zgrade Notr Dame?',
 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 'Šta je Grota u Notr Dami?',
 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?']

In [57]:
query_embeddings = []
all_query_embeddings = []

for query in data_mini['queries']:
    q_embeddings = []
    for q in query:
        q_embedding = get_sentence_embedding(q)
        q_embeddings.append([q_embedding])
    
    query_embeddings.append(q_embedding)
  
print(query_embeddings)
print(len(query_embeddings))
print(len(query_embeddings))
print(len(q_embedding))

[[0.32169026136398315, -0.15379604697227478, 0.10624294728040695, -0.1388511210680008, 0.03328621760010719, -0.08786018937826157, 0.002444535493850708, 0.19258356094360352, -0.049695294350385666, -0.054450057446956635, 0.039042409509420395, 0.029072165489196777, 0.10785575956106186, 0.06818115711212158, -0.0508132129907608, 0.22116610407829285, -0.10874566435813904, 0.264964759349823, -0.15767009556293488, -0.047454703599214554, 0.04242367669939995, -0.04477740451693535, 0.05614299327135086, 0.10436077415943146, 0.00859834160655737, -0.07210142910480499, 0.11305273324251175, 0.06546203792095184, -0.024945830926299095, 0.13248096406459808, -0.17989763617515564, -0.03549952432513237, 0.12168915569782257, 0.07786388695240021, -0.010019401088356972, 0.12410189211368561, 0.37668320536613464, -0.334774911403656, 0.009254632517695427, -0.022897910326719284, 0.013381878845393658, 0.0685216411948204, 0.13701029121875763, 0.11752776056528091, 0.1403501331806183, 0.004907190799713135, 0.091116935

In [59]:
print(len(query_embeddings[0]))


768


In [50]:
len(query_embeddings)

10

In [14]:
# from datasets import Dataset
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
# import pandas as pd
# import numpy as np

# # Example data (replace with your actual data)
# titles = ["University_of_Notre_Dame", "Normans"]
# contexts = [
#     "Arhitektonski, škola ima katolički karakter...",
#     "The Normans (Norman: Nourmands; French: Normands...)"
# ]
# questions = [
#     "Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?",
#     "In what country is Normandy located?"
# ]
# answers = [
#     {"text": ["Svetoj Bernadet Subiru"], "answer_start": [464]},
#     {"text": ["France"], "answer_start": [159]}
# ]

# # Prepare the dataset
# data_for_eval = {
#     'id': [str(i) for i in range(len(titles))],
#     'title': titles,
#     'context': contexts,
#     'question': questions,
#     'answers': answers
# }

# dataset_for_eval = Dataset.from_dict(data_for_eval)

# # Load model and tokenizer
# model_name = "classla/bcms-bertic"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# # Initialize the question-answering pipeline
# qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# # Generate predictions
# def generate_predictions(dataset):
#     predictions = []
#     for example in dataset:
#         prediction = qa_pipeline(question=example['question'], context=example['context'])
#         predictions.append({
#             'id': example['id'],
#             'prediction_text': prediction['answer'],
#             'no_answer_probability': prediction.get('score', 0.0)
#         })
#     return predictions

# predictions = generate_predictions(dataset_for_eval)

# # Custom evaluation metrics
# def compute_custom_metrics(predictions, references):
#     # Example metric: Exact Match (EM) and F1 Score
#     # def exact_match_score(prediction, ground_truth):
#     #     return int(prediction == ground_truth)

#     def f1_score(prediction, ground_truth):
#         prediction_tokens = prediction.split()
#         ground_truth_tokens = ground_truth.split()
#         common = set(prediction_tokens) & set(ground_truth_tokens)
#         if len(common) == 0:
#             return 0
#         precision = len(common) / len(prediction_tokens)
#         recall = len(common) / len(ground_truth_tokens)
#         f1 = 2 * (precision * recall) / (precision + recall)
#         return f1

#     em = []
#     f1 = []
#     for pred, ref in zip(predictions, references):
#         ground_truth_text = ref['text'][0]  # assuming there's only one ground truth answer
#         em.append(exact_match_score(pred['prediction_text'], ground_truth_text))
#         f1.append(f1_score(pred['prediction_text'], ground_truth_text))
    
#     return {
#         'exact_match': np.mean(em),
#         'f1_score': np.mean(f1)
#     }

# # References from the dataset
# references = [{'id': example['id'], 'text': example['answers']['text']} for example in dataset_for_eval]

# # Compute custom metrics
# results = compute_custom_metrics(predictions, references)

# print(results)


/home/selena/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'exact_match': np.float64(0.0), 'f1_score': np.float64(0.0)}


In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import numpy as np
from datasets import Dataset
import pytrec_eval


# Example data (replace with your actual data)
titles = ["University_of_Notre_Dame", "Normans"]
contexts = [
    "Arhitektonski, škola ima katolički karakter...",
    "The Normans (Norman: Nourmands; French: Normands...)"
]
questions = [
    "Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?",
    "In what country is Normandy located?"
]

# Assume these are the ground truth answers for evaluation purposes
reference_answers = [
    "Svetoj Bernadet Subiru",
    "France"
]

# Prepare the dataset
data_for_eval = {
    'id': [str(i) for i in range(len(titles))],
    'title': titles,
    'context': contexts,
    'question': questions
}

dataset_for_eval = Dataset.from_dict(data_for_eval)

# Load model and tokenizer
model_name = "classla/bcms-bertic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# Create model embeddings
# def generate_embeddings(text):
# Initialize the question-answering pipeline
# qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)
# Measuring query-context similarity
# def compute_similarity(query, context, top_k):
# Generate predictions
# def generate_predictions(dataset):
#     predictions = []
#     for example in dataset:
#         prediction = qa_pipeline(question=example['question'], context=example['context'])
#         predictions.append({
#             'id': example['id'],
#             'prediction_text': prediction['answer']
#         })
#     return predictions

# predictions = generate_predictions(dataset_for_eval)

# Custom evaluation metrics
def compute_custom_metrics(predictions, references):
    # Example metric: Exact Match (EM) and F1 Score
    def precision_k(prediction, ground_truth, k):
        evaluator = pytrec_eval.RelevanceEvaluator(ground_truth, {'P.4'})
        results = evaluator.evaluate(prediction)
        return results

    def f1_score(prediction, ground_truth):
        prediction_tokens = prediction.split()
        ground_truth_tokens = ground_truth.split()
        common = set(prediction_tokens) & set(ground_truth_tokens)
        if len(common) == 0:
            return 0
        precision = len(common) / len(prediction_tokens)
        recall = len(common) / len(ground_truth_tokens)
        f1 = 2 * (precision * recall) / (precision + recall)
        return f1

    em = []
    f1 = []
    for pred, ref in zip(predictions, references):
        em.append(exact_match_score(pred['prediction_text'], ref))
        f1.append(f1_score(pred['prediction_text'], ref))
    
    return {
        'exact_match': np.mean(em),
        'f1_score': np.mean(f1)
    }

# Compute custom metrics
results = compute_custom_metrics(predictions, reference_answers)

print(results)


/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'exact_match': np.float64(0.0), 'f1_score': np.float64(0.0)}
